In [15]:
import pdfplumber
import pandas as pd
from deep_translator import GoogleTranslator
import PyPDF2
import re
import operator
import numpy as np
    
pdf = pdfplumber.open("france.pdf")

def keywords():
    questions_to_keywords={
    "holiday":"Can your whole household afford to go for a week’s annual holiday, away from home?",
    "vegetarian":"Can your household afford a meal with meat, chicken, fish(or vegetarian equivalent)?",
    "expense": "Can your household afford an unexpected required expense(amount to be filled) and pay through its own resources?",
    "telephone":"Does your household have a telephone(fixed landline or mobile)?",
    "colour TV":"Does your household have a color TV?",
    "washing machine":"Does the household have a washing machine? ",
    "van":"Does your household have a car/van for private use? ",
    "dwelling":"Do you have any of the following problems with your dwelling/accommodation? ",
    "warm":"Can your household afford to keep its home adequately warm?  "
} 
    
    translated_keywords = []
    for key in questions_to_keywords.keys():
        translated = GoogleTranslator(source='en', target='french').translate(key)
        translated_keywords.append(translated)
    return translated_keywords

    
    
translated_keywords = keywords()
print(translated_keywords)

#testing regex - re.IGNORECASE works so it should work when matching keywords but doesnt for somre rason
def test_regex():
    matched = []
    matched.append(re.findall('hé',"héllo"))
    return matched


                               
#function to translate from detected language into english using deep translator library 
def translator(lines):
    translated_array =[]
    for i in lines:
        to_translate = i
        translated = GoogleTranslator(source='auto', target='en').translate(i)
        translated_array.append(translated)
    return translated_array


#cleans text removing next lines, whitespace, etc 
def clean(text):
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    
    return text



#DO NOT CHANGE- FULLY IDENTIFIES QUESTIONS ON A PAGE
#given a page of a docuent or list of pages, will find all questions on that page and translate into english 
def translate_document(pages):
    pdf1 = pdfplumber.open("france.pdf")
    translated_array = []
    #pages = list(pages)
    # writing page 161 will translate page 162
    pages= pages
    for number in pages:
        p1 = pdf1.pages[number]
        im = p1.to_image()
        text = p1.extract_text()
        text = clean(text)
        #text = re.split('[?]',text)  
        text = re.findall('(?<=[\?\.\!]\s)[^\?\n\.]+?\?',text)
        clean_sent  = []
        for sent in text:
            clean_sent.append(sent) 
        translated_array.append(clean_sent)
        translated_array.append(translator(clean_sent))
    return translated_array



pages = []
for word in translated_keywords:
    word = word.lower()
    pdf = PyPDF2.PdfFileReader(r"france.pdf")
    for i in range(0,pdf.getNumPages()):
        PageObj = pdf.getPage(i)
        Text = PageObj.extractText()     
        if re.findall(word,Text,re.IGNORECASE):
            #pages.append(word)
            pages.append(i) 
            
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list


d = {"col1": translate_document(pages)}
DftranslatedDoc=pd.DataFrame(data =d)
DftranslatedDoc

['vacances', 'végétarien', 'frais', 'Téléphone', 'téléviseur couleur', 'Machine à laver', 'van', 'logement', 'chaud']


,col1
0,[ La fiche-adresse a été créée par erreur Si ...
1,[The address sheet was created by mistake If R...
2,"[» si BS > 0 (Y14) Selon vous, le logement ..."
3,"[» if BS > 0 (Y14) According to you, is the ac..."
4,[ Sinon : TYPMEN Occupation du logement id...
...,...
317,[No NSP REFUSAL If TRAVAUX=2 and if V1WC=missi...
318,[Non NSP REFUS Combien payez-vous PAR AN...
319,[No DK REFUSAL How much do you pay PER YEAR fo...
320,"[ Confirmez-vous qu’actuellement, le montant a..."


In [8]:
  
'''
def translate_document(pages):
    
    pdf1 = pdfplumber.open("france.pdf")
    translated_array = []
    pages = pages

    for number in pages:
        p1 = pdf1.pages[i]
        im = p1.to_image()
        text = p1.extract_text()
        lines = text.split("\n")
        newlines = []
        for x in lines:
            if x.strip():
                newlines.append(x) 
        translated_array.append(translator(newlines))
    return translated_array
translate_document(pages)


'''


'\ndef translate_document(pages):\n    \n    pdf1 = pdfplumber.open("france.pdf")\n    translated_array = []\n    pages = pages\n\n    for number in pages:\n        p1 = pdf1.pages[i]\n        im = p1.to_image()\n        text = p1.extract_text()\n        lines = text.split("\n")\n        newlines = []\n        for x in lines:\n            if x.strip():\n                newlines.append(x) \n        translated_array.append(translator(newlines))\n    return translated_array\ntranslate_document(pages)\n\n\n'